In [1]:
from keras.models import load_model
import tensorflow as tf
import pydot
from tensorflow.keras.models import Sequential

In [2]:
import pydot_ng
import graphviz 

In [3]:
#pip install plot_model

In [4]:
#pip install graphviz

In [5]:
#pip install pydot

In [6]:
#pip install pydot_ng

In [7]:
from plot_model import plot_model
import PyPDF2
import re
import numpy as np

In [8]:
model = load_model('resume.h5')

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 100)          23233700  
                                                                 
 conv1d (Conv1D)             (None, 148, 128)          38528     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 49, 128)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 49, 128)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 47, 128)           49280     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                         

In [10]:
tf.keras.utils.plot_model(model, show_shapes = True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


## Text Extraction:

In [11]:
pdf = open('cv.pdf', 'rb') #resume input

In [12]:
pdfReader = PyPDF2.PdfReader(pdf)

In [13]:
pdfReader.metadata #About PDF

{'/Creator': 'Microsoft® Word 2016',
 '/CreationDate': "D:20230111110518+00'00'",
 '/Producer': 'www.ilovepdf.com',
 '/ModDate': 'D:20230111110518Z'}

In [14]:
x= (len(pdfReader.pages))
print("Number of pages in the document:",x)

Number of pages in the document: 2


In [15]:
text = " "

In [16]:
for page_number in range(x):
    pdfReader = PyPDF2.PdfReader(pdf)
    if pdfReader.is_encrypted:
        pdfReader.decrypt('')
    page = pdfReader.pages[page_number]
    text += page.extract_text()

In [17]:
text

' Brandon Thomas, Data Scientist  \n323 Knob Hill Apt. G  \nSan Francisco, CA 00000  \n415.555.1212 |  email@gmail.com  \nwww.dataminer.com  \n \nProfessional Profile  \nA former Ruby and Java programmer with newly acquired skills , an insatiable intellectual curiosity, and the \nability to mine hidden gems located within large sets of structured, semi -structured and unstructured data. \nAble to leverage a heavy dose of mathematics and applied statistics with visualization and a healthy  sense of \nexploration.  \nEducation  \nGrad Certificate, Data Mining  2012 University of California, San Diego  \nRelevant Courses:  Data Mining Methods and Techniques, Data Preparation for Data  \nMining and Advanced Methods and Applications  \nB.S. Computer Science  2009 San Francisco State University  \nCore Competencies  \nStrategic Thinking: Able to influence the strategic direction of the company by identifying opportunities in \nlarge, rich data sets and creating and implementing data driven s

In [18]:
pdf.close() # closing the pdf file object

## Pre Processing:

In [19]:
text = re.sub('http\S+\s*', ' ', text)  # remove URLs
text = re.sub('RT|cc', ' ', text)  # remove RT and cc
text = re.sub('#\S+', '', text)  # remove hashtags
text = re.sub('@\S+', '  ', text)  # remove mentions
text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', text)  # remove punctuation
text = re.sub(r'[^\x00-\x7f]',r' ', text) 
text = re.sub('\s+', ' ', text)  # remove extra whitespace

In [20]:
len(text)

3609

In [21]:
type(text)

str

In [22]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [23]:
stop_words = set(stopwords.words('english'))

In [24]:
#cachedStopWords = stopwords.words("english")
pattern = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*')
text = pattern.sub('', text)

In [25]:
type(text)

str

In [26]:
text=[text]

In [27]:
type(text)

list

In [28]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PRASHANT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [29]:
lemmatizer = WordNetLemmatizer()

In [30]:
text_out = ' '.join([lemmatizer.lemmatize(w) for w in text])
print(text_out)

 Brandon Thomas Data Scientist 323 Knob Hill Apt G San Francisco CA 00000 415 555 1212 email www dataminer com Professional Profile A former Ruby Java programmer newly acquired skills insatiable intellectual curiosity ability mine hidden gems located within large sets structured semi structured unstructured data Able leverage heavy dose mathematics applied statistics visualization healthy sense exploration Education Grad Certificate Data Mining 2012 University California San Diego Relevant Courses Data Mining Methods Techniques Data Preparation Data Mining Advanced Methods Applications B S Computer Science 2009 San Francisco State University Core Competencies Strategic Thinking Able influence strategic direction company identifying opportunities large rich data sets creating implementing data driven strategies fuel growth includin g revenue profits Modeling Design implement statistical predictive models cutting edge algorithms utilizing diverse sources data predict demand risk price el

In [31]:
text_out=[text_out]

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() # tokenize and build vocab
vectorizer.fit(text_out) # summarize
print(vectorizer.vocabulary_) # encode document
vector = vectorizer.transform(text_out).toarray() # summarize encoded vector

{'brandon': 36, 'thomas': 261, 'data': 73, 'scientist': 230, '323': 7, 'knob': 151, 'hill': 132, 'apt': 28, 'san': 224, 'francisco': 116, 'ca': 40, '00000': 0, '415': 8, '555': 9, '1212': 1, 'email': 102, 'www': 284, 'dataminer': 74, 'com': 51, 'professional': 200, 'profile': 201, 'former': 115, 'ruby': 222, 'java': 148, 'programmer': 203, 'newly': 177, 'acquired': 12, 'skills': 236, 'insatiable': 143, 'intellectual': 144, 'curiosity': 68, 'ability': 10, 'mine': 168, 'hidden': 130, 'gems': 120, 'located': 157, 'within': 282, 'large': 152, 'sets': 234, 'structured': 251, 'semi': 232, 'unstructured': 272, 'able': 11, 'leverage': 154, 'heavy': 128, 'dose': 93, 'mathematics': 164, 'applied': 25, 'statistics': 245, 'visualization': 281, 'healthy': 127, 'sense': 233, 'exploration': 110, 'education': 100, 'grad': 122, 'certificate': 44, 'mining': 169, '2012': 5, 'university': 270, 'california': 41, 'diego': 86, 'relevant': 212, 'courses': 66, 'methods': 167, 'techniques': 257, 'preparation': 

In [33]:
vector

array([[ 1,  1,  1,  1,  1,  2,  2,  1,  1,  1,  1,  2,  1,  1,  1,  1,
         1,  1,  4,  1,  2,  1,  2,  2,  3,  1,  1,  1,  1,  1,  1,  1,
         1,  3,  1,  1,  1,  1,  2,  1,  2,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  2,  1,  3,  1,  1,  1, 20,  1,  1,  1,  1,  1,  1,
         2,  1,  4,  2,  1,  1,  1,  1,  3,  2,  2,  1,  1,  1,  1,  2,
         1,  1,  2,  1,  2,  1,  1,  1,  1,  1,  2,  1,  1,  3,  1,  1,
         1,  1,  1,  1,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  1,
         1,  1,  1,  1,  1,  1,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  2,  1,  3,  2,  1,  1,  5,  1,  1,  1,  1,  1,  1,  2,
         1,  1,  2,  1,  1,  1,  1,  2,  1,  3,  1,  2,  1,  1,  1,  2,
         2,  1,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  2,  1,  1,  1,  2,  2,  1,  1,  2,  2,  2,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2, 

In [34]:
type(vector)

numpy.ndarray

In [35]:
vector.shape

(1, 285)

In [36]:
vector = vector[:,:150]

In [37]:
vector.shape

(1, 150)

In [38]:
tensor1 = tf.convert_to_tensor(vector)

In [39]:
tensor1.shape

TensorShape([1, 150])

In [40]:
tensor1

<tf.Tensor: shape=(1, 150), dtype=int64, numpy=
array([[ 1,  1,  1,  1,  1,  2,  2,  1,  1,  1,  1,  2,  1,  1,  1,  1,
         1,  1,  4,  1,  2,  1,  2,  2,  3,  1,  1,  1,  1,  1,  1,  1,
         1,  3,  1,  1,  1,  1,  2,  1,  2,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  2,  1,  3,  1,  1,  1, 20,  1,  1,  1,  1,  1,  1,
         2,  1,  4,  2,  1,  1,  1,  1,  3,  2,  2,  1,  1,  1,  1,  2,
         1,  1,  2,  1,  2,  1,  1,  1,  1,  1,  2,  1,  1,  3,  1,  1,
         1,  1,  1,  1,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  1,
         1,  1,  1,  1,  1,  1,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  2,  1,  3,  2]], dtype=int64)>

In [41]:
result = model.predict(vector)

1/1 [==============================] - 0s 168ms/step


In [42]:
result.shape

(1, 25)

In [43]:
result

array([[2.4272373e-08, 2.0306105e-08, 1.2299750e-04, 1.4073813e-08,
        1.8655915e-07, 2.8897065e-10, 4.9575250e-05, 5.3642876e-04,
        1.6318705e-02, 3.9368223e-07, 2.8893730e-08, 1.0516870e-10,
        2.2156970e-12, 9.8246574e-01, 1.9082706e-11, 4.5024793e-11,
        1.3256903e-12, 1.7043686e-07, 5.1313678e-09, 5.7558958e-10,
        4.5994835e-04, 4.5721848e-05, 3.1061095e-10, 1.3350341e-07,
        3.3940926e-09]], dtype=float32)

In [44]:
predicted_label_index = np.argmax(result)

In [45]:
predicted_label_index

13

In [46]:
np.max(result)

0.98246574